# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_file = ('../Output_data/Output_weather_data.csv')
vacation_file = pd.read_csv(data_file)
vacation_file.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness (%),Wind Speed (m/s),Country,Date
0,0,Clyde River,70.47,-68.59,44.60,87,40,8.05,CA,1596037471
1,1,Ürümqi,43.80,87.60,80.60,34,75,15.66,CN,1596037208
2,2,Bredasdorp,-34.53,20.04,66.20,55,0,13.87,ZA,1596037475
3,3,Puerto Madryn,-42.77,-65.04,47.77,52,0,11.83,AR,1596037477
4,4,Tasiilaq,65.61,-37.64,44.60,75,75,13.87,GL,1596037479


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
#Storing lat/lng into locations
locations = vacation_file[["Lat", "Lng"]].astype(float)

#Convert humidity to float and store
humid_locale = vacation_file["Humidity"].astype(float)

In [5]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humid_locale, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
over_75 = vacation_file[vacation_file['Max Temp']>=75]
over_75

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness (%),Wind Speed (m/s),Country,Date
1,1,Ürümqi,43.80,87.60,80.60,34,75,15.66,CN,1596037208
6,6,Moindou,-21.69,165.68,77.00,50,100,13.87,NC,1596037484
7,7,Korla,41.76,86.15,82.00,64,48,4.00,CN,1596037486
13,13,Muisne,0.60,-80.03,78.80,88,75,8.05,EC,1596037499
15,15,Santa Cruz del Sur,20.71,-78.00,88.92,65,3,14.56,CU,1596037503
...,...,...,...,...,...,...,...,...,...,...
596,596,Paphos,34.77,32.42,86.00,66,0,12.75,CY,1596038882
597,597,Singarāyakonda,15.25,80.03,86.59,69,4,9.33,IN,1596038885
599,599,Grand-Santi,4.25,-54.38,91.62,44,0,4.52,GF,1596038889
600,600,Dovers,12.88,-61.18,82.40,83,75,16.11,VC,1596038891


In [7]:
not_windy = over_75[over_75['Wind Speed (m/s)']<2]
not_windy

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness (%),Wind Speed (m/s),Country,Date
44,44,Kovin,44.75,20.98,93.00,38,0,1.12,RS,1596037567
124,124,Lucapa,-8.42,20.74,85.26,16,43,0.74,AO,1596037764
149,149,St. Pete Beach,27.73,-82.74,89.01,75,100,1.01,US,1596037825
165,165,Emmett,43.87,-116.50,78.01,46,0,1.99,US,1596037773
173,173,Auki,-8.77,160.70,78.26,80,88,0.74,SB,1596037877
185,185,Lodja,-3.48,23.43,90.79,21,0,1.68,CD,1596037904
197,197,Lázaro Cárdenas,17.96,-102.20,85.68,68,25,0.94,MX,1596037674
236,236,Gijang,35.24,129.21,75.20,94,75,1.12,KR,1596038028
272,272,Babu,24.42,111.52,81.07,74,93,1.97,CN,1596038114
352,352,Tabuk,17.42,121.44,75.25,87,100,0.34,PH,1596038309


In [8]:
hotel_df = not_windy[not_windy['Humidity']<=50]
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness (%),Wind Speed (m/s),Country,Date
44,44,Kovin,44.75,20.98,93.00,38,0,1.12,RS,1596037567
124,124,Lucapa,-8.42,20.74,85.26,16,43,0.74,AO,1596037764
165,165,Emmett,43.87,-116.50,78.01,46,0,1.99,US,1596037773
185,185,Lodja,-3.48,23.43,90.79,21,0,1.68,CD,1596037904
357,357,Gouré,13.98,10.27,95.00,42,11,1.99,NE,1596038323
360,360,King City,36.21,-121.13,75.99,35,0,1.99,US,1596038329


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
# Practice on one location

target_coordinates = "13.98, 10.27" 
target_search = "lodging"
target_radius = 50000
target_type = "lodging"

params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "types": target_type,
    "key": g_key
}

#API Call for nearest hotel
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

hotel_search = requests.get(base_url, params=params)
# print(hotel_search.url)
hotel_Goure = hotel_search.json()
# print(json.dumps(hotel_Goure, indent=4, sort_keys=True))

In [ ]:
# Practice on one location

target_coordinates = "36.21, -121.13" 
target_search = "lodging"
target_radius = 5000
target_type = "lodging"

params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "types": target_type,
    "key": g_key
}

#API Call for nearest hotel
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

hotel_search = requests.get(base_url, params=params)
print(hotel_search.url)
hotel_King = hotel_search.json()
# print(json.dumps(hotel_King, indent=4, sort_keys=True))

In [20]:
#Practice putting Hotel name in DF by df.loc method
hotel_df['Hotel Name'] = " "
hotel_df.loc[357, 'Hotel Name'] = hotel_Goure["results"][0]["name"]
hotel_df.loc[360, 'Hotel Name'] = hotel_King["results"][0]["name"]

C:\Users\brook\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
#create a new column called "Hotel Name"
# hotel_df['Hotel Name'] = " "

In [21]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness (%),Wind Speed (m/s),Country,Date,Hotel Name
44,44,Kovin,44.75,20.98,93.00,38,0,1.12,RS,1596037567,
124,124,Lucapa,-8.42,20.74,85.26,16,43,0.74,AO,1596037764,
165,165,Emmett,43.87,-116.50,78.01,46,0,1.99,US,1596037773,
185,185,Lodja,-3.48,23.43,90.79,21,0,1.68,CD,1596037904,
357,357,Gouré,13.98,10.27,95.00,42,11,1.99,NE,1596038323,OPVN
360,360,King City,36.21,-121.13,75.99,35,0,1.99,US,1596038329,Keefer's Inn


In [15]:
# find the closest Hotel of each type to coordinates (use class activity "Nearest Restaurant" as help)
hotels = []
count = 0

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
target_coordinates = (f' {lat}, {lng}') 
target_search = "lodging"
target_radius = 5000
target_type = "lodging"    
    
#Set up parameters
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "types": target_type,
    "key": g_key,
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# assemble url and make API request
    
hotel_search = requests.get(base_url, params=params)
hotel_data = hotel_search.json()
    
try:
    hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    count+=1
    print(f"There is a hotel close to {count}.")
        
        
except:
    count+=1
    print("You will not be vacationing at {count}.")
        
    print("------------")

There is a hotel close to 1.


C:\Users\brook\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=hotel_info, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

# Display figure
fig